# REST API Video Chunk Samples

## Objective
Sequential processing of video chunks in GPT-4 Turbo with Vision.	

## Time

You should expect to spend 5-10 minutes running this sample.

## Before you begin

#### Installation

In [ ]:
%pip install -r ../requirements.txt

### Parameters
You need to set a series of configurations such as GPT-4V_DEPLOYMENT_NAME, OPENAI_API_BASE, OPENAI_API_VERSION.

Add "OPENAI_API_KEY" as variable name and \<Your API Key Value\>as variable value in the environment variables.
 <br>
      
      WINDOWS Users: 
         setx OPENAI_API_KEY "REPLACE_WITH_YOUR_KEY_VALUE_HERE"

      MACOS/LINUX Users: 
         export OPENAI_API_KEY="REPLACE_WITH_YOUR_KEY_VALUE_HERE"


In [ ]:
# Setting up the deployment name
deployment_name: str = "<your GPT-4 Turbo with Vision deployment name>"
# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base: str = "<your resource base URL>"
# Currently OPENAI API have the following versions available: 2022-12-01.
# All versions follow the YYYY-MM-DD date structure.
openai_api_version: str = "<your OpenAI API version>"

# Insert your video SAS URL, e.g. https://<your-storage-account-name>.blob.core.windows.net/<your-container-name>/<your-video-name>?<SAS-token>
video_SAS_url = (
    "https://gpt4vsamples.blob.core.windows.net/videos/Redwire%20Field%20Trip%20-%203D%20Printing%20a%20Zune.mkv"
)

should_cleanup: bool = False

## Connect to your project
To start with let us create a config file with your project details. This file can be used in this sample or other samples to connect to your workspace.

In [ ]:
import json
from pathlib import Path

config = {
    "GPT-4V_DEPLOYMENT_NAME": deployment_name,
    "OPENAI_API_BASE": openai_api_base,
    "OPENAI_API_VERSION": openai_api_version,
}

p = Path("../config.json")

with p.open(mode="w") as file:
    file.write(json.dumps(config))

## Run this Example

In [ ]:
import time
import re
import sys

parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from shared_functions import call_GPT4V, sample_frames, download_video

### Call GPT-4 Turbo with Vision API

In [ ]:
from moviepy.editor import VideoFileClip

local_file_path = "downloaded_video.mp4"
assert download_video(video_SAS_url, local_file_path), "Failed to download video."
video = VideoFileClip(local_file_path)
video_length = video.duration

#  Call GPT-4 Turbo with Vision API on Each Video Chunk Sequentially
# Define the number of seconds for each segment
chunk_size = 120  # seconds

num_frames_to_sample = 20

if video_length is not None:
    print(f"Video Length: {video_length} seconds")
    sys_message = f"""
    The total length of the video is {video_length}s. Your job is to analyze a {chunk_size}-
    sec segment of the video and {num_frames_to_sample} frames from that segment. You will then provide a Current Scene Breakdown of the 
    video so far. Scenes must cover the entire video and non-overlapping. This breakdown should be a JSON object, with 
    each scenes being a key, and the value being an array of information about the scene, including topic, visual description,
    start and end times formatted MM:SS.
    """
    number_of_segments = int(video_length // chunk_size)
    updated_response = ""
    for i in range(number_of_segments + 1):  # Include the last segment
        start_time = i * chunk_size
        end_time = min((i + 1) * chunk_size, video_length)
        video_clip = video.subclip(start_time, end_time)

        user_prompt = f"How many scenes from {start_time}s to {end_time}s?"
        print(f"Segment {i+1}: {user_prompt}")
        if i > 0:
            user_prompt += f"""And here are scenes in the previous segments: {updated_response}. 
                            You need to combine the scenes in the previous segments with the scenes in this segment and provide a summary.
                            """
        base64_frames = sample_frames(video_clip, num_frames_to_sample)
        content = [{"type": "text", "text": user_prompt}]
        for frame in base64_frames:
            content.append({"type": "text", "text": f"Frame at time {start_time + frame[0]} seconds:"})
            content.append(
                {"type": "image_url", "image_url": {"url": f"data:image/jpg;base64,{frame[1]}", "detail": "low"}}
            )

        messages = [
            {"role": "system", "content": [{"type": "text", "text": sys_message}]},
            {"role": "user", "content": content},
        ]

        response = call_GPT4V(messages)
        updated_response = response["choices"][0]["message"]["content"]
        print(f"Response for segment {i+1}: {updated_response}")
        time.sleep(2)  # Avoid throttling

    # Print the final response
    sentences = re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", updated_response)
    for sentence in sentences:  # Print the content of the response
        print(sentence)
else:
    print("Failed to process video length.")

## Cleaning up

To clean up all Azure ML resources used in this example, you can delete the individual resources you created in this tutorial.

If you made a resource group specifically to run this example, you could instead [delete the resource group](https://learn.microsoft.com/en-us/azure/azure-resource-manager/management/delete-resource-group).

In [ ]:
if should_cleanup:
    Path(local_file_path).unlink()  # Delete the downloaded video
    # {{TODO: Add resource cleanup}}